In [1]:
from ml.nodes import *
from ml.traces import *
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
MNIST = Datasets('MNIST', mnist)
MNIST.info()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
                                Datasets:        0     N:-1 C:784 
Dataset: MNIST
  shape: N:-1 C:784 
  labels: 10
  test:       (10000, 784)
  validation: (5000, 784)
  train:      (55000, 784)


In [3]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [4]:
x = MNIST.x
y_ = MNIST.y_

In [5]:
x_im = Reshape(MNIST, 'C', 'WHC', (28, 28, 1))

c = Conv2d(x_im, 5, 32)
b = Bias(c)
relu = Relu(b)
mp1 = MaxPool(relu, 2, stride=2)

c2 = Conv2d(mp1, 5, 64)
b2 = Bias(c2)
relu2 = Relu(b2)
mp2 = MaxPool(relu2, 2, stride=2)

f3 = Flatten(mp2)
l3 = Linear(f3, 1024)
b3 = Bias(l3)
r3 = Relu(b3)

keep_prob = tf.placeholder(tf.float32)
d4 = Dropout(r3, keep_prob)
l4 = Linear(d4, 10)
b4 = Bias(l4)

s = SoftmaxCrossEntropyWithLogits(b4, y_)
sm = Softmax(b4)
ce = Mean(s)

acc = Accuracy(b4, y_)

                                 Reshape:        0     N:-1 W:28 H:28 C:1 
                                  Conv2d:      800     N:-1 W:28 H:28 C:32 
                                    Bias:       32     N:-1 W:28 H:28 C:32 
                                    Relu:        0     N:-1 W:28 H:28 C:32 
                                 MaxPool:        0     N:-1 W:14 H:14 C:32 
                                  Conv2d:    51200     N:-1 W:14 H:14 C:64 
                                    Bias:       64     N:-1 W:14 H:14 C:64 
                                    Relu:        0     N:-1 W:14 H:14 C:64 
                                 MaxPool:        0     N:-1 W:7 H:7 C:64 
                                 Flatten:        0     N:-1 C:3136 
                                  Linear:  3211264     N:-1 C:1024 
                                    Bias:     1024     N:-1 C:1024 
                                    Relu:        0     N:-1 C:1024 
                                 Dropout:      

In [6]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(ce.out())
sess.run(tf.global_variables_initializer())

In [ ]:
trackables = [ [ ScalarTrace(acc.out(), 'Accuracy',      yFn='log', skip=100),
                 ScalarTrace(ce.out(),  'Cross Entropy', yFn='log', skip=100), ],
               [ BiasTrace(b.b, 'Conv1 Bias'),  WeightTrace(c.W, 'Conv1 Weights'),   OutTrace(b.out(), 'Out')],
               [ BiasTrace(b2.b, 'Conv2 Bias'), WeightTrace(c2.W, 'Conv2 Weights'),  OutTrace(b2.out(), 'Out')],
               [ BiasTrace(b3.b, 'Lin1 Bias'),  WeightTrace(l3.W, 'Lin1 Weights'),   OutTrace(b3.out(), 'Out')],
               [ BiasTrace(b4.b, 'Lin2 Bias'),  WeightTrace(l4.W, 'Lin2 Weights'),   OutTrace(b4.out(), 'Out')],
               [ OutTrace(sm.out(), 'Softmax'),
                 TopTrace(sm.out(), 'Softmax/Top'),
                 TopTrace(sm.out(), 'Softmax/2nd', index=-2), ],
             ]

track = Track(trackables)

In [ ]:
def train(track, epochs=200):
    nodes = track.nodes()
    for i in range(epochs):
        for j in range(100):
            batch = mnist.train.next_batch(50)
            vals = sess.run([train_step] + nodes, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
            track.addBatch(vals[1:])
        r = sess.run(nodes, feed_dict={x: mnist.validation.images, y_: mnist.validation.labels, keep_prob: 1.0})
        print '%4d' % i,
        if (i+1) % 10 == 0:
            print '%8.04f' % (r[0]*100)
        track.addEpoch(r)
        
train(track)

   0    1    2    3    4    5    6    7    8    9  96.4400
  10   11   12   13   14   15   16   17   18   19  97.6400
  20   21   22   23   24   25   26   27   28   29  98.1200
  30   31   32   33   34   35   36   37   38   39  98.5400
  40   41

In [ ]:
track.plotAll(width=18, height=30)

In [ ]:
print("test accuracy %g" % 
          acc.out().eval(
            feed_dict={ x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))